<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [21]</a>'.</span>

In [1]:
# Parameters
alpha = 0.6
ratio = 0.1


In [2]:

import io
import os
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Arc
from IPython.display import display
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import mezzala
import itertools
import json
import urllib.request
from datetime import datetime  
from collections import defaultdict
from collections import Counter
from scipy import stats
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from PIL import Image
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import unidecode
import re
from pathlib import Path




In [3]:
# Definir la ruta relativa desde la ubicación actual (notebooks) hasta la carpeta de datos
data_path = "../../data_backup/archivos_parquet"

# Obtener la lista de archivos parquet que contienen "all"
parquet_files = [f for f in os.listdir(data_path) if f.endswith('.parquet') and 'all' in f.lower()]

# Procesar cada archivo y crear variables globales
for file in parquet_files:
    # Crear el nombre del DataFrame (reemplazar .parquet y agregar prefijo df_)
    df_name = 'df_' + file.replace('.parquet', '')
    
    # Leer el archivo parquet
    file_path = os.path.join(data_path, file)
    
    # Crear la variable global usando globals()
    globals()[df_name] = pd.read_parquet(file_path)
    
    # Imprimir información sobre el proceso
    print(f"Archivo procesado: {file}")
    print(f"Nombre del DataFrame: {df_name}")
    print(f"Dimensiones: {globals()[df_name].shape}")
    print("-" * 50)

Archivo procesado: seasons_teams_all.parquet
Nombre del DataFrame: df_seasons_teams_all
Dimensiones: (105, 7)
--------------------------------------------------
Archivo procesado: formations_league_all.parquet
Nombre del DataFrame: df_formations_league_all
Dimensiones: (40707, 9)
--------------------------------------------------
Archivo procesado: names_players_league_all.parquet
Nombre del DataFrame: df_names_players_league_all
Dimensiones: (9561, 12)
--------------------------------------------------
Archivo procesado: leagues_all.parquet
Nombre del DataFrame: df_leagues_all
Dimensiones: (42, 7)
--------------------------------------------------
Archivo procesado: sequence_data_league_all.parquet
Nombre del DataFrame: df_sequence_data_league_all
Dimensiones: (244675, 7)
--------------------------------------------------
Archivo procesado: players_league_all.parquet
Nombre del DataFrame: df_players_league_all
Dimensiones: (9584, 6)
--------------------------------------------------
A

In [4]:
print(df_player_stats_extended_league_all.columns)

Index(['goal', 'total_shot', 'shot_on_target', 'shot_off_target',
       'shot_blocked', 'shot_in_PA', 'shot_outside_of_PA', 'free_kick',
       'corner_kick', 'throw_in', 'penalty_kick', 'pass', 'pass_succeeded',
       'pass_failed', 'assist', 'key_pass', 'control_under_pressure', 'cross',
       'cross_succeeded', 'final_third_area_pass',
       'final_third_area_pass_succeeded', 'middle_area_pass',
       'middle_area_pass_succeeded', 'defensive_area_pass',
       'defensive_area_pass_succeeded', 'short_pass', 'short_pass_succeeded',
       'medium_range_pass', 'medium_range_pass_succeeded', 'long_pass',
       'long_pass_succeeded', 'forward_pass', 'forward_pass_succeeded',
       'backward_pass', 'backward_pass_succeeded', 'sideways_pass',
       'sideways_pass_succeeded', 'tackle', 'tackle_succeeded', 'aerial_duel',
       'aerial_duel_succeeded', 'aerial_duel_failed', 'ground_duel',
       'ground_duel_succeeded', 'ground_duel_failed', 'loose_ball_duel',
       'loose_ball_duel

In [5]:
print(df_names_players_league_all.columns)

Index(['player_id', 'root_player_id', 'player_name', 'player_last_name',
       'player_name_en', 'player_last_name_en', 'back_number', 'player_role',
       'main_position', 'team_id', 'birth_date', 'season_id'],
      dtype='object')


In [6]:

def find_league_name(row, df_league_all):
    season_id = str(row['season_id'])
    
    for _, league_row in df_league_all.iterrows():
        # Maneja tanto strings de listas como de arrays numpy
        season_ids_str = league_row['season_ids']
        try:
            # Si es una representación string de una lista normal
            season_ids = ast.literal_eval(season_ids_str)
        except ValueError:
            # Si es una representación string de array numpy
            if 'array(' in season_ids_str:
                # Extrae números del string del array
                numbers = season_ids_str.replace('array([', '').replace('])', '').split(',')
                season_ids = [int(num.strip()) for num in numbers]
            else:
                continue
        
        if season_id in map(str, season_ids):
            return league_row['name']
    
    return None


# Paso 1: Realizar el merge inicial
df_combined = pd.merge(
    df_player_stats_extended_league_all,
    df_names_players_league_all[['player_id', 'player_name_en', 'player_last_name_en', 'main_position', 'back_number']],
    on='player_id',
    how='left'
)

# Paso 1B: Función para formatear nombres (primera letra mayúscula, resto minúscula)
def format_name(name):
    if pd.isna(name) or name == '0':
        return name
    return ' '.join(word.capitalize() for word in name.split())

# Paso 1C: Aplicar formato a los nombres
df_combined['player_name_en'] = df_combined['player_name_en'].apply(format_name)
df_combined['player_last_name_en'] = df_combined['player_last_name_en'].apply(format_name)

# Paso 1D: Función para buscar nombres en df_events_league_all
def find_name(row, events_df):
    if pd.isna(row['player_name_en']) or row['player_name_en'] == '0' or pd.isna(row['player_last_name_en']) or row['player_last_name_en'] == '0':
        player_id = row.name  # Asumiendo que el índice es el player_id
        match = events_df[events_df['player_id'] == player_id]
        if not match.empty:
            return pd.Series({
                'player_name_en': format_name(match['player_name'].iloc[0]) if 'player_name' in match.columns else row['player_name_en'],
                'player_last_name_en': format_name(match['player_last_name'].iloc[0]) if 'player_last_name' in match.columns else row['player_last_name_en']
            })
    return row[['player_name_en', 'player_last_name_en']]

# Paso 1E: Aplicar la función de búsqueda
df_combined[['player_name_en', 'player_last_name_en']] = df_combined.apply(lambda row: find_name(row, df_events_league_all), axis=1)

# Paso 2: Merge el resultado con df_teams_league_all
df_combined = pd.merge(
    df_combined,
    df_teams_league_all[['team_id', 'name']],
    on='team_id',
    how='left'
)

# Renombrar la columna 'name' a 'team_name' para mayor claridad
df_combined = df_combined.rename(columns={'name': 'team_name'})

# Paso 3: Aplicar la función para encontrar el nombre de la liga
df_combined['league_name'] = df_combined.apply(lambda row: find_league_name(row, df_leagues_all), axis=1)

# Paso 4: Merge con df_matches_all para añadir información adicional
df_combined = pd.merge(
    df_combined,
    df_matches_league_all[['match_id', 'home_team_id', 'away_team_id', 'home_team_name', 'away_team_name', 'round_name', 'season_name']],
    on='match_id',
    how='left'
)

# Paso 5: Crear columnas para el nombre del equipo y condición de local/visitante
df_combined['equipo_partido'] = np.where(
    df_combined['team_id'] == df_combined['home_team_id'],
    df_combined['home_team_name'],
    df_combined['away_team_name']
)

df_combined['home_away'] = np.where(
    df_combined['team_id'] == df_combined['home_team_id'],
    'local',
    'visitante'
)

# Nuevo Paso 6: Merge con df_lineups_league_all y crear la columna starter_sub
df_combined = pd.merge(
    df_combined,
    df_lineups_league_all[['match_id', 'season_id', 'player_id', 'is_starting_lineup']],
    on=['match_id', 'season_id', 'player_id'],
    how='left'
)

df_combined['starter_sub'] = np.where(
    df_combined['is_starting_lineup'] == True,
    'starter',
    'sub'
)

# Reemplazar valores nulos por 0 (excepto en starter_sub)
for column in df_combined.columns:
    if column != 'starter_sub':
        df_combined[column] = df_combined[column].fillna(0)

# Eliminar filas duplicadas
df_combined = df_combined.drop_duplicates()

# Verificar las columnas del DataFrame final
df_combined

/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/4215001531.py:110: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined[column] = df_combined[column].fillna(0)


,goal,total_shot,shot_on_target,shot_off_target,shot_blocked,shot_in_PA,shot_outside_of_PA,free_kick,corner_kick,throw_in,...,home_team_id,away_team_id,home_team_name,away_team_name,round_name,season_name,equipo_partido,home_away,is_starting_lineup,starter_sub
0,0,3,1,2,0,3,0,4,0,0,...,7897,7939,Deportivo Alavés Cad. A,SD Eibar Cad. A,Cuartos de final,22/23,Deportivo Alavés Cad. A,local,True,starter
12,1,2,1,1,0,2,0,0,0,0,...,7897,7939,Deportivo Alavés Cad. A,SD Eibar Cad. A,Cuartos de final,22/23,Deportivo Alavés Cad. A,local,True,starter
24,0,0,0,0,0,0,0,1,0,0,...,7897,7939,Deportivo Alavés Cad. A,SD Eibar Cad. A,Cuartos de final,22/23,Deportivo Alavés Cad. A,local,False,sub
36,0,1,0,1,0,1,0,0,0,0,...,7897,7939,Deportivo Alavés Cad. A,SD Eibar Cad. A,Cuartos de final,22/23,Deportivo Alavés Cad. A,local,True,starter
48,1,4,2,1,1,4,0,0,0,0,...,7897,7939,Deportivo Alavés Cad. A,SD Eibar Cad. A,Cuartos de final,22/23,Deportivo Alavés Cad. A,local,True,starter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82645,0,0,0,0,0,0,0,2,0,0,...,7753,15447,SCD Durango,Deportivo Alavés C,Jornada 21,24/25,Deportivo Alavés C,visitante,False,sub
82648,1,2,1,1,0,1,1,0,0,2,...,7753,15447,SCD Durango,Deportivo Alavés C,Jornada 21,24/25,Deportivo Alavés C,visitante,True,starter
82651,0,0,0,0,0,0,0,0,0,6,...,7753,15447,SCD Durango,Deportivo Alavés C,Jornada 21,24/25,Deportivo Alavés C,visitante,True,starter
82654,0,0,0,0,0,0,0,0,0,0,...,7753,15447,SCD Durango,Deportivo Alavés C,Jornada 21,24/25,Deportivo Alavés C,visitante,False,sub


In [7]:
# Obtener los valores únicos de la columna 'main_position'
df_combined['season_name'].unique()

array(['22/23', '21/22', '23/24', '24/25'], dtype=object)

In [8]:

# Lista de columnas a mantener
columnas_a_mantener = [
    'goal', 'match_id','total_shot', 'shot_on_target', 'shot_in_PA', 'shot_outside_of_PA',
    'free_kick', 'penalty_kick', 'corner_kick', 'pass', 'pass_succeeded',
    'assist', 'key_pass', 'control_under_pressure', 'cross',
    'cross_succeeded', 'final_third_area_pass', 'final_third_area_pass_succeeded',
    'middle_area_pass', 'middle_area_pass_succeeded', 'defensive_area_pass',
    'defensive_area_pass_succeeded', 'long_pass', 'long_pass_succeeded',
    'forward_pass', 'forward_pass_succeeded', 'backward_pass',
    'backward_pass_succeeded', 'sideways_pass', 'sideways_pass_succeeded',
    'tackle', 'tackle_succeeded', 'aerial_duel', 'aerial_duel_succeeded',
    'ground_duel', 'ground_duel_succeeded', 'loose_ball_duel',
    'loose_ball_duel_succeeded', 'block', 'intercept', 'clearance', 'recovery',
    'intervention', 'take_on', 'take_on_succeeded', 'mistake',
    'foul', 'offside', 'yellow_card', 'red_card', 'own_goal',
    'goal_conceded', 'aerial_clearance', 'aerial_clearance_succeeded',
    'defensive_line_support', 'defensive_line_support_succeeded',
    'rating', 'save_by_catching', 'save_by_punching','play_time'
]

# Cambiar los nombres de las columnas del nombre del jugador
df_combined = df_combined.rename(columns={
    'player_name_en': 'player_name',
    'player_last_name_en': 'player_last_name'
})
# Crear la nueva columna 'match'
df_combined['match'] = df_combined['home_team_name'] + '-' + df_combined['away_team_name']

# Añadir las columnas que agregamos en los pasos anteriores
columnas_adicionales = ['player_id', 'player_name', 'player_last_name', 'back_number', 'main_position', 'team_name', 'league_name', 'season_name', 'round_name', 'match','home_away', 'starter_sub']

# Combinar todas las columnas que queremos mantener
todas_columnas = columnas_a_mantener + columnas_adicionales

# Seleccionar solo estas columnas de df_combined y filtrar por defensas centrales
df_players = df_combined[todas_columnas]

# Convertir play_time de milésimas de segundos a minutos enteros
df_players['play_time_minutes'] = np.round(df_players['play_time'] / (1000 * 60)).astype(int)

# Contar el número de valores nulos por columna antes de reemplazarlos
null_counts_before = df_combined.isnull().sum()
print("Número de valores nulos por columna antes de reemplazarlos:")
print(null_counts_before)
print("\n")

# Reemplazar valores nulos por 0 (excepto en back_number)
for column in df_combined.columns:
    if column != 'back_number':
        df_combined[column] = df_combined[column].fillna(0)
        

# Mostrar información sobre el DataFrame
df_players

Número de valores nulos por columna antes de reemplazarlos:
goal                  0
total_shot            0
shot_on_target        0
shot_off_target       0
shot_blocked          0
                     ..
equipo_partido        0
home_away             0
is_starting_lineup    0
starter_sub           0
match                 0
Length: 96, dtype: int64




/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/281883632.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_players['play_time_minutes'] = np.round(df_players['play_time'] / (1000 * 60)).astype(int)


,goal,match_id,total_shot,shot_on_target,shot_in_PA,shot_outside_of_PA,free_kick,penalty_kick,corner_kick,pass,...,back_number,main_position,team_name,league_name,season_name,round_name,match,home_away,starter_sub,play_time_minutes
0,0,93857,3,1,3,0,4,0,0,30,...,10,CM,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,starter,80
12,1,93857,2,1,2,0,0,0,0,21,...,11,LW,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,starter,80
24,0,93857,0,0,0,0,1,0,0,3,...,17,CM,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,sub,16
36,0,93857,1,0,1,0,0,0,0,26,...,6,CDM,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,starter,70
48,1,93857,4,2,4,0,0,0,0,14,...,9,CAM,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,starter,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82645,0,165032,0,0,0,0,2,0,0,16,...,17,CM,Deportivo Alavés C,0,24/25,Jornada 21,SCD Durango-Deportivo Alavés C,visitante,sub,30
82648,1,165032,2,1,1,1,0,1,0,18,...,16,RW,Deportivo Alavés C,0,24/25,Jornada 21,SCD Durango-Deportivo Alavés C,visitante,starter,90
82651,0,165032,0,0,0,0,0,0,0,18,...,12,RB,Deportivo Alavés C,0,24/25,Jornada 21,SCD Durango-Deportivo Alavés C,visitante,starter,90
82654,0,165032,0,0,0,0,0,0,0,9,...,2,CF,Deportivo Alavés C,0,24/25,Jornada 21,SCD Durango-Deportivo Alavés C,visitante,sub,51


In [9]:

df_players['player'] = df_players['player_name'].astype(str) + ' ' + df_players['player_last_name'].astype(str)
df_players.fillna(0, inplace=True)
df_players  =df_players[(df_players['main_position'] != 0) & (df_players['player'] != '0 0')]

df_players 

/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/1983768074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_players['player'] = df_players['player_name'].astype(str) + ' ' + df_players['player_last_name'].astype(str)
/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/1983768074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_players.fillna(0, inplace=True)


,goal,match_id,total_shot,shot_on_target,shot_in_PA,shot_outside_of_PA,free_kick,penalty_kick,corner_kick,pass,...,main_position,team_name,league_name,season_name,round_name,match,home_away,starter_sub,play_time_minutes,player
0,0,93857,3,1,3,0,4,0,0,30,...,CM,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,starter,80,Unax Ariza Fernandez
12,1,93857,2,1,2,0,0,0,0,21,...,LW,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,starter,80,Biniam Gonzalez
24,0,93857,0,0,0,0,1,0,0,3,...,CM,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,sub,16,Aitor Suberviola
36,0,93857,1,0,1,0,0,0,0,26,...,CDM,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,starter,70,Alejandro Dolera Lopez
48,1,93857,4,2,4,0,0,0,0,14,...,CAM,Deportivo Alavés Cad. A,0,22/23,Cuartos de final,Deportivo Alavés Cad. A-SD Eibar Cad. A,local,starter,76,Raul Perez Cortes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82645,0,165032,0,0,0,0,2,0,0,16,...,CM,Deportivo Alavés C,0,24/25,Jornada 21,SCD Durango-Deportivo Alavés C,visitante,sub,30,Mikel Relloso Martinez
82648,1,165032,2,1,1,1,0,1,0,18,...,RW,Deportivo Alavés C,0,24/25,Jornada 21,SCD Durango-Deportivo Alavés C,visitante,starter,90,Inaki Alberca Laquidain
82651,0,165032,0,0,0,0,0,0,0,18,...,RB,Deportivo Alavés C,0,24/25,Jornada 21,SCD Durango-Deportivo Alavés C,visitante,starter,90,Manex Irizar Landaribar
82654,0,165032,0,0,0,0,0,0,0,9,...,CF,Deportivo Alavés C,0,24/25,Jornada 21,SCD Durango-Deportivo Alavés C,visitante,sub,51,Ela Mangue Noe


In [10]:
df_players ['season_name'].unique()

array(['22/23', '21/22', '23/24', '24/25'], dtype=object)

In [11]:


def create_kpi_players(df_players):
    print("Iniciando la creación de df_KPI_players...")
    print("Columnas disponibles en df_players:")
    print(df_players.columns)
    
    required_columns = ['match_id', 'match','team_name', 'league_name', 'season_name', 'round_name',
                        'home_away','player_id', 'player', 'main_position', 'back_number', 'starter_sub', 'play_time_minutes']
    
    missing_columns = [col for col in required_columns if col not in df_players.columns]
    if missing_columns:
        print("Columnas faltantes:")
        print(missing_columns)
        raise ValueError(f"Las siguientes columnas no están presentes en df_players: {missing_columns}")
    
    df_KPI_players = df_players[required_columns].copy()
    
    df_KPI_players['participacion'] = df_KPI_players.apply(lambda row:
        (10 if row['play_time_minutes'] >= 90 else
         0 if row['play_time_minutes'] == 0 else
         (row['play_time_minutes'] / 90) * 10) +
        (3 if row['starter_sub'] == 'starter' else 0),
        axis=1
    ).round(2)
    
    def safe_divide(a, b):
        return np.round(np.where(b != 0, a / b, 0), 2)
    
    df_KPI_players['%_tiros_dentro_area'] = safe_divide(df_players['shot_in_PA'], df_players['total_shot'])
    df_KPI_players['%_tiros_fuera_area'] = safe_divide(df_players['shot_outside_of_PA'], df_players['total_shot'])
    df_KPI_players['Capacidad_disparos'] = df_KPI_players['%_tiros_fuera_area'] +  df_KPI_players['%_tiros_dentro_area']
    
    df_KPI_players['lanzador_ABP'] = (df_players['free_kick'] +  df_players['corner_kick'] + df_players['penalty_kick']).round(2)
    df_KPI_players['precision_pase'] = safe_divide(df_players['pass_succeeded'], df_players['pass'])
    df_KPI_players['influencia_pases_gol'] = (df_players['assist'] + df_players['key_pass']+ df_players['cross_succeeded']).round(2)
    
    df_KPI_players['controles_bajo_presion'] = df_players['control_under_pressure'].round(2)

    
    df_KPI_players['Eficacia_disparos'] = safe_divide(df_players['shot_on_target'], df_players['total_shot'])
    df_KPI_players['Eficacia_goleadora'] = safe_divide(df_players['goal'], df_players['shot_on_target'])
    df_KPI_players['Habilidad_regate']= safe_divide(df_players['take_on_succeeded'], df_players['take_on'])
                                   
    df_KPI_players['influencia_finalización'] = safe_divide(df_players['final_third_area_pass_succeeded'], df_players['final_third_area_pass'])
    
    df_KPI_players['influencia_construccion'] = safe_divide(
        (df_players['middle_area_pass_succeeded'] + df_players['defensive_area_pass_succeeded']),
        (df_players['middle_area_pass'] + df_players['defensive_area_pass'])
    )
    df_KPI_players['precision_pases_largos'] = safe_divide(df_players['long_pass_succeeded'], df_players['long_pass'])
    df_KPI_players['verticalidad_pase'] = (
        ((df_players['forward_pass'] - (df_players['backward_pass'] + df_players['sideways_pass'])) * df_players['forward_pass_succeeded'])
    )
    
    df_KPI_players['talento'] = (((df_KPI_players['Habilidad_regate'] + df_players['shot_on_target']/df_players['total_shot']) + (df_players['control_under_pressure']) + ((df_players['free_kick'] +  df_players['corner_kick'] + df_players['penalty_kick'])) +
                                 (df_players['pass_succeeded']) + (df_players['goal'])+ (df_players['assist'] + df_players['key_pass']+ df_players['cross_succeeded']) +
                                 (df_players['take_on_succeeded']) + (df_players['tackle_succeeded']) + (df_players['aerial_duel_succeeded']) + (df_players['ground_duel_succeeded']) + 
                                 (df_players['loose_ball_duel_succeeded']) + (df_players['rating']) + (df_players['forward_pass_succeeded'])+ (df_players['save_by_catching'] + df_players['save_by_punching'])) - 
                                 (df_players['mistake'] + df_players['foul']+ df_players['own_goal']+ df_players['goal_conceded']+ df_players['offside']))

    df_KPI_players['efectividad_entradas'] = safe_divide(df_players['tackle_succeeded'], df_players['tackle'])
    df_KPI_players['efectividad_juego_aereo'] = safe_divide(df_players['aerial_duel_succeeded'], df_players['aerial_duel'])
    df_KPI_players['efectividad_duelos_suelo'] = safe_divide(df_players['ground_duel_succeeded'], df_players['ground_duel'])
    df_KPI_players['efectividad_balones_divididos'] = safe_divide(df_players['loose_ball_duel_succeeded'], df_players['loose_ball_duel'])
    df_KPI_players['contundencia'] = (df_KPI_players['efectividad_balones_divididos'] + 
                                      df_KPI_players['efectividad_duelos_suelo'] + 
                                      df_KPI_players['efectividad_juego_aereo'] + 
                                      df_KPI_players['efectividad_entradas']).round(2)
    df_KPI_players['capacidad_defensiva'] = safe_divide((df_players['intercept'] + df_players['clearance']+ df_players['tackle'] + df_players['block']+ df_players['ground_duel']+ df_players['loose_ball_duel'] +
                                                         df_players['recovery'] + df_players['save_by_catching'] + df_players['save_by_punching']), df_players['intervention'])
    df_KPI_players['capacidad_juego_aereo_def'] = safe_divide(safe_divide(df_players['aerial_clearance_succeeded'], df_players['aerial_clearance']), df_players['clearance'])
    df_KPI_players['rendimiento_posicional_def'] = safe_divide(df_players['defensive_line_support_succeeded'], df_players['defensive_line_support'])
    
    df_KPI_players['calificacion_Bepro'] = df_players['rating'].round(2)
    df_KPI_players['errores'] = df_players['mistake'].round(2)
    df_KPI_players['faltas'] = df_players['foul'].round(2)
    df_KPI_players['fuera_juego'] = df_players['offside'].round(2)
    df_KPI_players['tarj_amarilla'] = df_players['yellow_card'].round(2)
    df_KPI_players['tarj_roja'] = df_players['red_card'].round(2)
    df_KPI_players['gol_en_propia'] = df_players['own_goal'].round(2)
    df_KPI_players['gol_en_contra'] = df_players['goal_conceded'].round(2)
    
    # Redondear todas las columnas numéricas a 2 decimales
    df_KPI_players = df_KPI_players.round(2)
    
    print("df_KPI_players creado exitosamente dentro de la función.")
    return df_KPI_players

if 'df_players' not in globals():
    print("Error: 'df_players' no está definido. Asegúrate de cargar tu DataFrame antes de ejecutar esta función.")
else:
    try:
        print("Intentando crear df_KPI_players...")
        df_KPI_players = create_kpi_players(df_players)
        print("df_KPI_players creado exitosamente.")
        print("\nPrimeras filas de df_KPI_players:")
        print(df_KPI_players.head())
        print("\nInformación de df_KPI_players:")
        print(df_KPI_players.info())
    except ValueError as e:
        print(f"\nError al crear df_KPI_players: {e}")
        print("Por favor, verifica las columnas en df_players y asegúrate de que todas las columnas necesarias estén presentes.")
    except Exception as e:
        print(f"\nOcurrió un error inesperado: {e}")

if 'df_KPI_players' in globals():
    print("\ndf_KPI_players existe en el espacio de trabajo global")
    print(f"Forma de df_KPI_players: {df_KPI_players.shape}")
else:
    print("\ndf_KPI_players no se creó correctamente o no está en el espacio global.")
    print("Asegúrate de que no hay errores en la ejecución de la función y que 'df_players' contiene los datos correctos.")


Intentando crear df_KPI_players...
Iniciando la creación de df_KPI_players...
Columnas disponibles en df_players:
Index(['goal', 'match_id', 'total_shot', 'shot_on_target', 'shot_in_PA',
       'shot_outside_of_PA', 'free_kick', 'penalty_kick', 'corner_kick',
       'pass', 'pass_succeeded', 'assist', 'key_pass',
       'control_under_pressure', 'cross', 'cross_succeeded',
       'final_third_area_pass', 'final_third_area_pass_succeeded',
       'middle_area_pass', 'middle_area_pass_succeeded', 'defensive_area_pass',
       'defensive_area_pass_succeeded', 'long_pass', 'long_pass_succeeded',
       'forward_pass', 'forward_pass_succeeded', 'backward_pass',
       'backward_pass_succeeded', 'sideways_pass', 'sideways_pass_succeeded',
       'tackle', 'tackle_succeeded', 'aerial_duel', 'aerial_duel_succeeded',
       'ground_duel', 'ground_duel_succeeded', 'loose_ball_duel',
       'loose_ball_duel_succeeded', 'block', 'intercept', 'clearance',
       'recovery', 'intervention', 'take_on

In [12]:


def calculate_kpis(df):
    # Verificar las columnas existentes
    existing_columns = df.columns.tolist()
    
    # Función de normalización (1 9)
    def normalize(data):
        min_val = np.min(data)
        max_val = np.max(data)
        if min_val == max_val:
            return np.full_like(data, 0.1)
        normalized = 0.1 + 1 * (data - min_val) / (max_val - min_val)
        return np.clip(normalized, 0.1, 1).round(2)

    kpi_columns = []

    # KPI 1: Condicional (Participacion y parametros fisicos)
    if all(col in existing_columns for col in ['participacion', 'capacidad_defensiva',
       'capacidad_juego_aereo_def']):
        condicional = (df['participacion']* 0.4+
                        df['capacidad_defensiva'] *0.3+ 
                        df['capacidad_juego_aereo_def'] * 0.3)
        df['KPI_condicional'] = (normalize(condicional) * 9.5).round(2)
        kpi_columns.append('KPI_condicional')

    # KPI 2: Construcción de Ataque
    if all(col in existing_columns for col in ['influencia_construccion', 'precision_pase',  'precision_pases_largos', 'verticalidad_pase', 'participacion']):
        construccion_ataque = ((df['influencia_construccion'] * 0.4 +
                               df['precision_pase']* 0.2 +
                               df['precision_pases_largos'] * 0.2 +
                               df['verticalidad_pase'] * 0.2) * df['participacion'])
        df['KPI_construccion_ataque'] = (normalize(construccion_ataque) * 9.5).round(2)
        kpi_columns.append('KPI_construccion_ataque')

    # KPI 3: Progresión
    if all(col in existing_columns for col in ['verticalidad_pase', 'precision_pase', 'precision_pases_largos', 'participacion']):
        progresion = ((df['verticalidad_pase'] * 0.6 +
                      df['precision_pase']* 0.2 +
                      df['precision_pases_largos'] * 0.2) * df['participacion'])
        df['KPI_progresion'] = (normalize(progresion) * 9.5).round(2)
        kpi_columns.append('KPI_progresion')

    # KPI 4: Habilidad Individual
    if all(col in existing_columns for col in ['talento', 'controles_bajo_presion', 'participacion']):
        habilidad_individual = ((df['talento'] * 0.8) + 
                              (df['controles_bajo_presion']) * 0.2 * df['participacion'])
        df['KPI_habilidad_individual'] = (normalize(habilidad_individual) * 9.5).round(2)
        kpi_columns.append('KPI_habilidad_individual')

    # KPI 5: Peligro Generado
    if all(col in existing_columns for col in ['influencia_pases_gol', 'participacion', 'Capacidad_disparos']):
        peligro_generado = ((df['influencia_pases_gol'] * 0.6 )+
                            (df['Capacidad_disparos'] * 0.4) * df['participacion'])
        df['KPI_peligro_generado'] = (normalize(peligro_generado)  * 9.5).round(2)
        kpi_columns.append('KPI_peligro_generado')

    # KPI 6: Finalización
    if all(col in existing_columns for col in ['Eficacia_disparos', 'Eficacia_goleadora', 'influencia_finalización', 'participacion']):
        finalizacion = (((df['Eficacia_disparos'] + df['Eficacia_goleadora'])* 0.6 +
                        df['influencia_finalización'] * 0.4) * df['participacion'])
        df['KPI_finalizacion'] = (normalize(finalizacion) * 9.5).round(2)
        kpi_columns.append('KPI_finalizacion') 

    # KPI 7: Eficacia Defensiva
    if all(col in existing_columns for col in ['efectividad_entradas', 'capacidad_defensiva', 'rendimiento_posicional_def', 'participacion', 'faltas']):
        eficacia_defensiva = (((df['efectividad_entradas'] - df['faltas']) * 0.3 +
                              df['capacidad_defensiva'] * 0.4 +
                              df['rendimiento_posicional_def'] * 0.3) * df['participacion'])
        df['KPI_eficacia_defensiva'] = (normalize(eficacia_defensiva) * 9.5).round(2)
        kpi_columns.append('KPI_eficacia_defensiva') 

    # KPI 8: Juego Aéreo
    if all(col in existing_columns for col in ['efectividad_juego_aereo', 'capacidad_juego_aereo_def', 'participacion']):
        juego_aereo = ((df['efectividad_juego_aereo'] * 0.7 +
                       df['capacidad_juego_aereo_def'] * 0.3) * df['participacion']) 
        df['KPI_juego_aereo'] = (normalize(juego_aereo) * 9.5).round(2)
        kpi_columns.append('KPI_juego_aereo')  

    # KPI 9: Capacidad de Recuperación
    if all(col in existing_columns for col in ['capacidad_defensiva', 'efectividad_balones_divididos', 'participacion']):
        capacidad_recuperacion = ((df['capacidad_defensiva'] * 0.7 +
                                  df['efectividad_balones_divididos'] * 0.3) * df['participacion'])
        df['KPI_capacidad_recuperacion'] = (normalize(capacidad_recuperacion) * 9.5).round(2)
        kpi_columns.append('KPI_capacidad_recuperacion')

    # KPI 10: Posicionamiento Táctico
    if all(col in existing_columns for col in ['rendimiento_posicional_def', 'fuera_juego', 'capacidad_defensiva', 'participacion']):
        posicionamiento_tactico = ((df['rendimiento_posicional_def'] * 0.5 +
                                   (1 - df['fuera_juego'] / df['participacion'].replace(0, np.nan)) * 0.3 +
                                   df['capacidad_defensiva'] * 0.2) * df['participacion']) 
        df['KPI_posicionamiento_tactico'] = (normalize(posicionamiento_tactico)  * 9.5).round(2)
        kpi_columns.append('KPI_posicionamiento_tactico')

    # KPI Rendimiento (suma promedio de los KPIs calculados, previa ponderacion por posicion)
    if kpi_columns:
        df['KPI_rendimiento'] = df[kpi_columns].mean(axis=1).round(2)
        kpi_columns.append('KPI_rendimiento')

    return df

# Aplicar la función al DataFrame
df_KPI_players = calculate_kpis(df_KPI_players)

# Verificar las columnas KPI que se calcularon
kpi_columns = [col for col in df_KPI_players.columns if col.startswith('KPI_')]

if kpi_columns:
    # Crear un ranking de los 20 mejores jugadores basado en KPI_rendimiento
    if 'KPI_rendimiento' in kpi_columns:
        top_20_players = df_KPI_players.nlargest(20, 'KPI_rendimiento')
        
        # Imprimir el ranking
        print("Ranking de los 20 mejores jugadores basado en KPI_rendimiento:")
        columns_to_show = ['player', 'team_name', 'main_position', 'league_name',
       'season_name', 'round_name', 'match'] + kpi_columns
        print(top_20_players[columns_to_show].to_string(index=False))

        # Mostrar estadísticas descriptivas de los KPIs
        print("\nEstadísticas descriptivas de los KPIs:")
        print(df_KPI_players[kpi_columns].describe())

        # Verificar si hay algún NaN en los resultados
        print("\nNúmero de NaN en cada KPI:")
        print(df_KPI_players[kpi_columns].isna().sum())
        
    else:
        print("KPI_rendimiento no se pudo calcular.")
else:
    print("No se pudieron calcular KPIs debido a la falta de columnas necesarias.")

# Imprimir las columnas disponibles en el DataFrame para diagnóstico
print("\nColumnas disponibles en el DataFrame:")

print(df_KPI_players.columns.tolist())


Ranking de los 20 mejores jugadores basado en KPI_rendimiento:
                 player                team_name main_position  league_name season_name round_name                                           match  KPI_condicional  KPI_construccion_ataque  KPI_progresion  KPI_habilidad_individual  KPI_peligro_generado  KPI_finalizacion  KPI_eficacia_defensiva  KPI_juego_aereo  KPI_capacidad_recuperacion  KPI_posicionamiento_tactico  KPI_rendimiento
        Rafael Martinez  Deportivo Alavés Juv. A            LB            0       23/24 Jornada 28      Deportivo Alavés Juv. A-SCD Durango Juv. A             9.40                     4.75            4.75                      7.22                  4.66              6.84                    9.22             7.60                        9.12                         9.12             7.27
  Joseba Perez Escudero      Antiguoko KE Juv. A            CB            0       23/24 Jornada 25     Antiguoko KE Juv. A-Deportivo Alavés Juv. A             9.50  

In [13]:
# Verificar las columnas KPI que se calcularon
kpi_columns = [col for col in df_KPI_players.columns if col.startswith('KPI_')]

kpi_columns

['KPI_condicional',
 'KPI_construccion_ataque',
 'KPI_progresion',
 'KPI_habilidad_individual',
 'KPI_peligro_generado',
 'KPI_finalizacion',
 'KPI_eficacia_defensiva',
 'KPI_juego_aereo',
 'KPI_capacidad_recuperacion',
 'KPI_posicionamiento_tactico',
 'KPI_rendimiento']

In [14]:
df_KPI_players.fillna(0, inplace=True)
df_KPI_players  =df_KPI_players[(df_KPI_players['main_position'] != 0) & (df_KPI_players['player'] != '0 0')]

In [15]:
df_KPI_players.sort_values(by='KPI_rendimiento', ascending=False)

,match_id,match,team_name,league_name,season_name,round_name,home_away,player_id,player,main_position,...,KPI_construccion_ataque,KPI_progresion,KPI_habilidad_individual,KPI_peligro_generado,KPI_finalizacion,KPI_eficacia_defensiva,KPI_juego_aereo,KPI_capacidad_recuperacion,KPI_posicionamiento_tactico,KPI_rendimiento
13492,132645,Deportivo Alavés Juv. A-SCD Durango Juv. A,Deportivo Alavés Juv. A,0,23/24,Jornada 28,local,302138,Rafael Martinez,LB,...,4.75,4.75,7.22,4.66,6.84,9.22,7.60,9.12,9.12,7.27
14825,130614,Antiguoko KE Juv. A-Deportivo Alavés Juv. A,Antiguoko KE Juv. A,0,23/24,Jornada 25,local,475368,Joseba Perez Escudero,CB,...,5.89,5.89,3.14,4.28,8.07,9.50,7.03,9.50,9.50,7.23
40245,162870,SD Eibar Cad. A-Deportivo Alavés Cad. A,SD Eibar Cad. A,0,24/25,Jornada 18,local,544532,Iker Artetxe Belaustegi,CM,...,6.65,6.65,3.61,4.75,5.32,9.31,5.22,9.31,9.31,6.96
15861,127985,UDC Txantrea KKE Juv. A-Deportivo Alavés Juv. A,Deportivo Alavés Juv. A,0,23/24,Jornada 23,visitante,302049,Igor Martinez,CB,...,5.98,5.98,0.00,1.71,3.32,9.50,6.84,9.50,9.50,6.87
6922,83532,Deportivo Alavés Juv. A-CD Tudelano Juv. A,Deportivo Alavés Juv. A,0,22/23,Jornada 18,local,367389,Inaki Saenz,CM,...,5.51,5.51,4.75,8.55,6.84,7.41,7.60,7.03,7.12,6.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72741,123651,Deportivo Alavés C-CD San Ignacio,CD San Ignacio,0,23/24,Jornada 19,visitante,328912,Benat Leon Berganzo,CAM,...,5.89,5.89,1.04,1.04,0.95,1.71,0.95,0.95,0.95,2.03
58696,154105,CD Subiza-CD Anguiano,CD Anguiano,0,24/25,Jornada 8,visitante,541538,Aaron Camara,CF,...,5.89,5.89,1.14,0.95,0.95,1.71,0.95,0.95,0.95,2.03
73940,121492,SD Eibar B-CD Padura,SD Eibar B,0,23/24,Jornada 16,local,175331,Mikel Zabala Ojeda,CM,...,5.89,5.89,1.04,0.95,0.95,1.71,0.95,0.95,0.95,2.02
10988,77771,Antiguoko KE Juv. A-Deportivo Alavés Juv. A,Deportivo Alavés Juv. A,0,22/23,Jornada 5,visitante,302140,Gaizka Garcia,GK,...,3.61,3.61,0.00,0.95,0.95,1.71,0.95,0.95,1.24,1.94


In [16]:
# Verifica las posiciones únicas en la columna correspondiente
posiciones_unicas = df_KPI_players['main_position'].unique()
print(posiciones_unicas)

# Definir todas las posiciones posibles
#main_position= ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']

# Elegir una posición (puedes cambiar este valor sin modificar el resto del código)
#main_position = 'CAM'  # Cambia este valor para elegir una posición distinta


['CM' 'LW' 'CDM' 'CAM' 'CF' 'RW' 'CB' 'LB' 'RB' 'GK' 'LF' 'RF' 'RM' 'LWB'
 'RWB' 'LM']


In [17]:

# Filtrar el DataFrame eliminando las filas donde 'main_position' y 'player' es igual a 0
#df_KPI_players.copy

#  Dataframe base df_KPI_players (todos los jugadores/todos los equipos)

In [18]:
# Primero convertimos back_number a tipo string
df_KPI_players['back_number'] = df_KPI_players['back_number'].astype(str)

# Convertimos columnas numéricas potencialmente problemáticas
columnas_numericas = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
                     'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
                     'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
                     'KPI_posicionamiento_tactico', 'KPI_rendimiento', 'play_time_minutes']

# Convertir columnas numéricas a float
for col in columnas_numericas:
    df_KPI_players[col] = pd.to_numeric(df_KPI_players[col], errors='coerce')

# Convertir columnas categóricas a string
columnas_categoricas = ['player', 'player_id', 'main_position', 'team_name', 'league_name', 
                       'season_name', 'round_name', 'match', 'match_id', 'home_away', 'starter_sub']
for col in columnas_categoricas:
    df_KPI_players[col] = df_KPI_players[col].astype(str)

# Guardar en formato parquet
df_KPI_players.to_parquet('../../data_backup/archivos_parquet/df_KPI_players.parquet', index=False)

### Dataframe base df_KPI_players_position (añadimos ponderacion de KPI por posicion,todos los jugadores/todos los equipos)

In [19]:
import pandas as pd
import numpy as np
from pathlib import Path

# Definir todas las posiciones posibles
main_position = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']

# Definir las ponderaciones para cada posición
pesos_main_position = {
    'GK': {'KPI_condicional': 2.85, 'KPI_construccion_ataque': 0.1, 'KPI_progresion': 0.025,
           'KPI_habilidad_individual': 2.4, 'KPI_peligro_generado': 0.45, 'KPI_finalizacion': 0.45,
           'KPI_eficacia_defensiva': 2.97, 'KPI_juego_aereo': 2.94, 'KPI_capacidad_recuperacion': 2.7,
           'KPI_posicionamiento_tactico': 2.85},
    
    'CB': {'KPI_condicional': 2.85, 'KPI_construccion_ataque': 0.15, 'KPI_progresion': 0.075,
           'KPI_habilidad_individual': 2.55, 'KPI_peligro_generado': 0.9, 'KPI_finalizacion': 0.6,
           'KPI_eficacia_defensiva': 2.97, 'KPI_juego_aereo': 2.94, 'KPI_capacidad_recuperacion': 2.85,
           'KPI_posicionamiento_tactico': 2.97},

    'RB': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.2, 'KPI_progresion': 0.275,
           'KPI_habilidad_individual': 2.7, 'KPI_peligro_generado': 2.4, 'KPI_finalizacion': 1.5,
           'KPI_eficacia_defensiva': 2.55, 'KPI_juego_aereo': 2.1, 'KPI_capacidad_recuperacion': 2.7,
           'KPI_posicionamiento_tactico': 2.7},

    'LB': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.2, 'KPI_progresion': 0.275,
           'KPI_habilidad_individual': 2.7, 'KPI_peligro_generado': 2.4, 'KPI_finalizacion': 1.5,
           'KPI_eficacia_defensiva': 2.55, 'KPI_juego_aereo': 2.1, 'KPI_capacidad_recuperacion': 2.7,
           'KPI_posicionamiento_tactico': 2.7},

    'RWB': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.375,
            'KPI_habilidad_individual': 2.85, 'KPI_peligro_generado': 2.7, 'KPI_finalizacion': 1.8,
            'KPI_eficacia_defensiva': 2.4, 'KPI_juego_aereo': 2.1, 'KPI_capacidad_recuperacion': 2.7,
            'KPI_posicionamiento_tactico': 2.7},

    'LWB': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.375,
            'KPI_habilidad_individual': 2.85, 'KPI_peligro_generado': 2.7, 'KPI_finalizacion': 1.8,
            'KPI_eficacia_defensiva': 2.4, 'KPI_juego_aereo': 2.1, 'KPI_capacidad_recuperacion': 2.7,
            'KPI_posicionamiento_tactico': 2.7},

    'CDM': {'KPI_condicional': 2.85, 'KPI_construccion_ataque': 0.275, 'KPI_progresion': 0.325,
            'KPI_habilidad_individual': 2.7, 'KPI_peligro_generado': 2.25, 'KPI_finalizacion': 1.35,
            'KPI_eficacia_defensiva': 2.91, 'KPI_juego_aereo': 2.7, 'KPI_capacidad_recuperacion': 2.91,
            'KPI_posicionamiento_tactico': 2.97},

    'CM': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.35, 'KPI_progresion': 0.35,
           'KPI_habilidad_individual': 2.85, 'KPI_peligro_generado': 2.7, 'KPI_finalizacion': 2.1,
           'KPI_eficacia_defensiva': 2.4, 'KPI_juego_aereo': 2.1, 'KPI_capacidad_recuperacion': 2.55,
           'KPI_posicionamiento_tactico': 2.7},

    'CAM': {'KPI_condicional': 2.55, 'KPI_construccion_ataque': 0.375, 'KPI_progresion': 0.35,
            'KPI_habilidad_individual': 2.97, 'KPI_peligro_generado': 2.91, 'KPI_finalizacion': 2.7,
            'KPI_eficacia_defensiva': 1.8, 'KPI_juego_aereo': 1.5, 'KPI_capacidad_recuperacion': 2.1,
            'KPI_posicionamiento_tactico': 2.7},

    'RM': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.325,
           'KPI_habilidad_individual': 2.91, 'KPI_peligro_generado': 2.85, 'KPI_finalizacion': 2.55,
           'KPI_eficacia_defensiva': 1.8, 'KPI_juego_aereo': 1.5, 'KPI_capacidad_recuperacion': 2.4,
           'KPI_posicionamiento_tactico': 2.55},

    'LM': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.325,
           'KPI_habilidad_individual': 2.91, 'KPI_peligro_generado': 2.85, 'KPI_finalizacion': 2.55,
           'KPI_eficacia_defensiva': 1.8, 'KPI_juego_aereo': 1.5, 'KPI_capacidad_recuperacion': 2.4,
           'KPI_posicionamiento_tactico': 2.55},

    'RW': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.325,
           'KPI_habilidad_individual': 2.91, 'KPI_peligro_generado': 2.85, 'KPI_finalizacion': 2.7,
           'KPI_eficacia_defensiva': 1.5, 'KPI_juego_aereo': 1.5, 'KPI_capacidad_recuperacion': 2.4,
           'KPI_posicionamiento_tactico': 2.55},

    'LW': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.325,
           'KPI_habilidad_individual': 2.91, 'KPI_peligro_generado': 2.85, 'KPI_finalizacion': 2.7,
           'KPI_eficacia_defensiva': 1.5, 'KPI_juego_aereo': 1.5, 'KPI_capacidad_recuperacion': 2.4,
           'KPI_posicionamiento_tactico': 2.55},

    'LF': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.325,
           'KPI_habilidad_individual': 2.91, 'KPI_peligro_generado': 2.91, 'KPI_finalizacion': 2.85,
           'KPI_eficacia_defensiva': 1.35, 'KPI_juego_aereo': 2.1, 'KPI_capacidad_recuperacion': 2.1,
           'KPI_posicionamiento_tactico': 2.55},

    'RF': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.325,
           'KPI_habilidad_individual': 2.91, 'KPI_peligro_generado': 2.91, 'KPI_finalizacion': 2.85,
           'KPI_eficacia_defensiva': 1.35, 'KPI_juego_aereo': 2.1, 'KPI_capacidad_recuperacion': 2.1,
           'KPI_posicionamiento_tactico': 2.55},

    'CF': {'KPI_condicional': 2.7, 'KPI_construccion_ataque': 0.25, 'KPI_progresion': 0.25,
           'KPI_habilidad_individual': 2.97, 'KPI_peligro_generado': 2.97, 'KPI_finalizacion': 2.97,
           'KPI_eficacia_defensiva': 1.2, 'KPI_juego_aereo': 2.55, 'KPI_capacidad_recuperacion': 1.8,
           'KPI_posicionamiento_tactico': 2.55}
}

try:
    # Obtener la ruta base del proyecto
    notebook_path = Path().absolute()
    base_path = notebook_path.parent.parent  # Subir dos niveles para llegar a Bepro
    
    # Construir las rutas para los archivos
    input_parquet = base_path / 'data_backup' / 'archivos_parquet' / 'df_KPI_players.parquet'
    output_parquet = base_path / 'data_backup' / 'archivos_parquet' / 'df_KPI_players_bueno.parquet'
    
    # Crear el directorio si no existe
    output_parquet.parent.mkdir(parents=True, exist_ok=True)
    
    # Debug info
    print(f"Ruta de entrada: {input_parquet}")
    print(f"Ruta de salida: {output_parquet}")
    
    # Leer el archivo parquet
    df_KPI_players = pd.read_parquet(input_parquet)
    
    # Definir columnas KPI
    kpi_columns = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
                   'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
                   'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
                   'KPI_posicionamiento_tactico']
    
    def normalizar_kpis_por_liga(grupo):
        for kpi in kpi_columns:
            grupo[kpi] = (grupo[kpi] - grupo[kpi].mean()) / grupo[kpi].std() * 2.25 + 4.5
            grupo[kpi] = grupo[kpi].clip(0, 9)  # Limitar valores entre 0 y 9
        return grupo
    
    # Aplicar normalización por liga
    print("Normalizando KPIs por liga...")
    df_KPI_players = df_KPI_players.groupby('match_id').apply(normalizar_kpis_por_liga)
    
    # Calcular rendimiento
    def calcular_rendimiento(row):
        posicion = row['main_position']
        peso = pesos_main_position.get(posicion, {})
        if not peso:
            print(f"Advertencia: Posición no encontrada: {posicion}")
            return np.nan
        return sum(row[kpi] * peso.get(kpi, 0) for kpi in kpi_columns)
    
    print("Calculando KPI rendimiento...")
    df_KPI_players['KPI_rendimiento'] = df_KPI_players[kpi_columns].mean(axis=1)
    
    # Normalizar rendimiento por liga
    def normalizar_rendimiento_por_liga(grupo):
        if len(grupo) > 0:  # Verificar que el grupo no esté vacío
            min_val = grupo['KPI_rendimiento'].min()
            max_val = grupo['KPI_rendimiento'].max()
            if max_val > min_val:  # Evitar división por cero
                grupo['KPI_rendimiento'] = (grupo['KPI_rendimiento'] - min_val) / (max_val - min_val) * 10
        return grupo
    
    print("Normalizando rendimiento por liga...")
    df_KPI_players = df_KPI_players.groupby('league_name').apply(normalizar_rendimiento_por_liga)
    
    # Seleccionar columnas finales y ordenar
    columnas_finales = ['back_number', 'player', 'player_id', 'main_position', 
                        'team_name', 'league_name', 'season_name', 'round_name', 
                        'match', 'match_id', 'home_away', 'starter_sub', 
                        'play_time_minutes'] + kpi_columns + ['KPI_rendimiento']
    
    df_KPI_players_position = df_KPI_players[columnas_finales].sort_values(
        by='KPI_rendimiento', ascending=False
    ).round(2)
    
    # Guardar el resultado
    print("Guardando resultado...")
    df_KPI_players_position.to_parquet(output_parquet, index=False)
    
    print("¡Proceso completado!")
    print(f"\nPrimeras 25 filas del DataFrame resultante:")
    print(df_KPI_players_position.head(25))

except FileNotFoundError as e:
    print(f"Error: No se pudo encontrar el archivo: {e.filename}")
    print(f"Directorio actual: {Path().absolute()}")
except Exception as e:
    print(f"Error inesperado: {str(e)}")

Ruta de entrada: /Users/imac/Programas/Datos_API_Bepro/Alaves_Github/BePro/data_backup/archivos_parquet/df_KPI_players.parquet
Ruta de salida: /Users/imac/Programas/Datos_API_Bepro/Alaves_Github/BePro/data_backup/archivos_parquet/df_KPI_players_bueno.parquet
Normalizando KPIs por liga...


Calculando KPI rendimiento...
Normalizando rendimiento por liga...
Guardando resultado...
¡Proceso completado!

Primeras 25 filas del DataFrame resultante:
                           back_number                         player  \
league_name match_id                                                    
0           161922   8117            3           Alvaro Mateo Montiel   
            156907   6230            5           Iker Domingo Ruperez   
            162870   5976           15        Iker Artetxe Belaustegi   
            119723   11936          37                    Xanet Olaiz   
            118431   5671            8  Anartz Abraldes Azkorbebeitia   
            131133   11232           6          Marcos Cena Fernandez   
            121450   5595            4       Luken Beobide De La Mata   
            53823    10405           8             Imanol Mingo Munoz   
            52971    583            12        Miguel Barandalla Ostiz   
            113216   2988            3   

/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/2079432942.py:124: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_KPI_players = df_KPI_players.groupby('match_id').apply(normalizar_kpis_por_liga)
/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/2079432942.py:148: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_KPI_players = df_KPI_players.groupby('league_name').apply(normali

### Obtenemos valores muy parecidos en ambos df lo que nos indica que la ponderacion de metricas ha sido acertada con una aproximacion del 99,2%

## KPIs Posicion & Metricas Avanzadas,  para ver los mejores "Rendimiento General, (todos jug./todos equipos)"


In [20]:

# Definir todas las posiciones posibles
main_position = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF']

# Definir las ponderaciones para cada posición

pesos_main_position = {
    'GK': {'KPI_condicional': 0.8, 'KPI_construccion_ataque': 0.25, 'KPI_progresion': 0.1,
           'KPI_habilidad_individual': 0.3, 'KPI_peligro_generado': 0.1, 'KPI_finalizacion': 0.1,
           'KPI_eficacia_defensiva': 0.95, 'KPI_juego_aereo': 0.85, 'KPI_capacidad_recuperacion': 0.7,
           'KPI_posicionamiento_tactico': 0.8},
    
    'CB': {'KPI_condicional': 0.8, 'KPI_construccion_ataque': 0.35, 'KPI_progresion': 0.2,
           'KPI_habilidad_individual': 0.35, 'KPI_peligro_generado': 0.2, 'KPI_finalizacion': 0.15,
           'KPI_eficacia_defensiva': 0.95, 'KPI_juego_aereo': 0.9, 'KPI_capacidad_recuperacion': 0.85,
           'KPI_posicionamiento_tactico': 0.95},

    'RB': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.45, 'KPI_progresion': 0.6,
           'KPI_habilidad_individual': 0.75, 'KPI_peligro_generado': 0.5, 'KPI_finalizacion': 0.3,
           'KPI_eficacia_defensiva': 0.65, 'KPI_juego_aereo': 0.45, 'KPI_capacidad_recuperacion': 0.75,
           'KPI_posicionamiento_tactico': 0.7},

    'LB': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.45, 'KPI_progresion': 0.6,
           'KPI_habilidad_individual': 0.75, 'KPI_peligro_generado': 0.5, 'KPI_finalizacion': 0.3,
           'KPI_eficacia_defensiva': 0.65, 'KPI_juego_aereo': 0.45, 'KPI_capacidad_recuperacion': 0.75,
           'KPI_posicionamiento_tactico': 0.7},

    'RWB': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.65, 'KPI_progresion': 0.8,
            'KPI_habilidad_individual': 0.8, 'KPI_peligro_generado': 0.75, 'KPI_finalizacion': 0.4,
            'KPI_eficacia_defensiva': 0.6, 'KPI_juego_aereo': 0.45, 'KPI_capacidad_recuperacion': 0.75,
            'KPI_posicionamiento_tactico': 0.7},

    'LWB': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.65, 'KPI_progresion': 0.8,
            'KPI_habilidad_individual': 0.8, 'KPI_peligro_generado': 0.75, 'KPI_finalizacion': 0.4,
            'KPI_eficacia_defensiva': 0.6, 'KPI_juego_aereo': 0.45, 'KPI_capacidad_recuperacion': 0.75,
            'KPI_posicionamiento_tactico': 0.7},

    'CDM': {'KPI_condicional': 0.8, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.7,
            'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.55, 'KPI_finalizacion': 0.3,
            'KPI_eficacia_defensiva': 0.9, 'KPI_juego_aereo': 0.75, 'KPI_capacidad_recuperacion': 0.9,
            'KPI_posicionamiento_tactico': 0.95},

    'CM': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.75, 'KPI_progresion': 0.75,
           'KPI_habilidad_individual': 0.8, 'KPI_peligro_generado': 0.7, 'KPI_finalizacion': 0.45,
           'KPI_eficacia_defensiva': 0.55, 'KPI_juego_aereo': 0.45, 'KPI_capacidad_recuperacion': 0.65,
           'KPI_posicionamiento_tactico': 0.7},

    'CAM': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.8, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.95, 'KPI_peligro_generado': 0.9, 'KPI_finalizacion': 0.75,
            'KPI_eficacia_defensiva': 0.35, 'KPI_juego_aereo': 0.25, 'KPI_capacidad_recuperacion': 0.35,
            'KPI_posicionamiento_tactico': 0.7},

    'RM': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.65, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.75, 'KPI_finalizacion': 0.65,
           'KPI_eficacia_defensiva': 0.35, 'KPI_juego_aereo': 0.25, 'KPI_capacidad_recuperacion': 0.45,
           'KPI_posicionamiento_tactico': 0.6},

    'LM': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.65, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.75, 'KPI_finalizacion': 0.65,
           'KPI_eficacia_defensiva': 0.35, 'KPI_juego_aereo': 0.25, 'KPI_capacidad_recuperacion': 0.45,
           'KPI_posicionamiento_tactico': 0.6},

    'RW': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.65, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.75, 'KPI_finalizacion': 0.65,
           'KPI_eficacia_defensiva': 0.35, 'KPI_juego_aereo': 0.25, 'KPI_capacidad_recuperacion': 0.45,
           'KPI_posicionamiento_tactico': 0.6},

    'LW': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.65, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.75, 'KPI_finalizacion': 0.65,
           'KPI_eficacia_defensiva': 0.35, 'KPI_juego_aereo': 0.25, 'KPI_capacidad_recuperacion': 0.45,
           'KPI_posicionamiento_tactico': 0.6},

    'LF': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.65, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.75, 'KPI_finalizacion': 0.65,
           'KPI_eficacia_defensiva': 0.35, 'KPI_juego_aereo': 0.25, 'KPI_capacidad_recuperacion': 0.45,
           'KPI_posicionamiento_tactico': 0.6},

    'RF': {'KPI_condicional': 0.7, 'KPI_construccion_ataque': 0.65, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.75, 'KPI_finalizacion': 0.65,
           'KPI_eficacia_defensiva': 0.35, 'KPI_juego_aereo': 0.25, 'KPI_capacidad_recuperacion': 0.45,
           'KPI_posicionamiento_tactico': 0.6},

    'CF': {'KPI_condicional': 0.75, 'KPI_construccion_ataque': 0.55, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.95, 'KPI_peligro_generado': 0.95, 'KPI_finalizacion': 0.95,
           'KPI_eficacia_defensiva': 0.25, 'KPI_juego_aereo': 0.65, 'KPI_capacidad_recuperacion': 0.35,
           'KPI_posicionamiento_tactico': 0.6}
}


# Leer el archivo CSV (ajusta el nombre del archivo según sea necesario)
df_KPI_players = pd.read_parquet('../../data_backup/archivos_parquet/df_KPI_players.parquet')

# Elegir la posición
posicion_elegida = "LB"  # Aquí puedes definir la posición directamente

# Filtrar el DataFrame por la posición elegida
df_KPI_players_posicion = df_KPI_players[df_KPI_players['main_position'] == posicion_elegida]

# Normalizar KPIs
kpi_columns = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
               'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
               'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
               'KPI_posicionamiento_tactico']

# Normalizar los KPI a un rango de 0 a 9.5
df_KPI_players_posicion[kpi_columns] = df_KPI_players_posicion[kpi_columns].apply(
    lambda x: (x - x.min()) / (x.max() - x.min()) * 10)

# Calcular el rendimiento ponderado total
def calcular_rendimiento(row):
    peso = pesos_main_position[posicion_elegida]
    rendimiento_total = sum(row[kpi] * peso[kpi] for kpi in kpi_columns)
    return rendimiento_total

df_KPI_players_posicion['KPI_rendimiento'] = df_KPI_players_posicion.apply(calcular_rendimiento, axis=1)
df_KPI_players_posicion['KPI_rendimiento'] = df_KPI_players_posicion[kpi_columns].mean(axis=1)

# Seleccionar columnas finales
df_KPI_players = df_KPI_players_posicion[['back_number', 'player', 'player_id', 'main_position', 
                                           'team_name', 'league_name', 'season_name', 'round_name', 
                                           'match', 'match_id', 'home_away', 'starter_sub', 
                                           'play_time_minutes', 'KPI_condicional', 
                                           'KPI_construccion_ataque', 'KPI_progresion', 
                                           'KPI_habilidad_individual', 'KPI_peligro_generado', 
                                           'KPI_finalizacion', 'KPI_eficacia_defensiva', 
                                           'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
                                           'KPI_posicionamiento_tactico', 'KPI_rendimiento']]

# Mostrar el DataFrame final
df_KPI_players_posicion = df_KPI_players.sort_values(by='KPI_rendimiento', ascending=False).round(2)
df_KPI_players_posicion

/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/1169984129.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_KPI_players_posicion[kpi_columns] = df_KPI_players_posicion[kpi_columns].apply(
/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/1169984129.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_KPI_players_posicion['KPI_rendimiento'] = df_KPI_players_posicion.apply(calcular_rendimiento, axis=1)
/var/folders/cm/p9cytb0d0vnfhdcstzncydc80000gn/T/ipykernel_9988/11

,back_number,player,player_id,main_position,team_name,league_name,season_name,round_name,match,match_id,...,KPI_construccion_ataque,KPI_progresion,KPI_habilidad_individual,KPI_peligro_generado,KPI_finalizacion,KPI_eficacia_defensiva,KPI_juego_aereo,KPI_capacidad_recuperacion,KPI_posicionamiento_tactico,KPI_rendimiento
1721,3,Rafael Martinez,302138,LB,Deportivo Alavés Juv. A,0,23/24,Jornada 28,Deportivo Alavés Juv. A-SCD Durango Juv. A,132645,...,3.67,3.67,7.83,5.42,6.89,10.00,10.00,10.00,10.00,7.75
8117,3,Alvaro Mateo Montiel,543984,LB,Union Deportiva Logroñes,0,24/25,Jornada 19,Union Deportiva Logroñes-Deportivo Alavés B,161922,...,7.47,7.47,6.59,5.42,6.67,6.89,10.00,6.98,7.39,7.27
380,12,Miguel Barandalla Ostiz,221140,LB,Athletic Club Juv. A,0,21/22,Jornada 28,Athletic Club Juv. A-AD San Juan Juv. A,54897,...,5.19,5.19,6.07,9.44,9.78,6.30,10.00,6.16,6.78,7.23
7870,3,Eneko Ortiz,274397,LB,Deportivo Alavés B,0,23/24,Jornada 26,Deportivo Alavés B-C.D. Valle de Egües,107515,...,4.80,4.80,7.72,5.42,10.00,7.73,3.29,7.67,8.03,6.79
2988,3,Rafael Martinez,302138,LB,Deportivo Alavés Juv. A,0,23/24,Jornada 5,Deportivo Alavés Juv. A-SD Eibar Juv. A,113216,...,6.32,6.32,4.12,5.98,6.89,7.37,6.71,7.67,8.03,6.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4415,3,Aaron Sanz Pereda,356727,LB,Deportivo Alavés C,0,22/23,Jornada 30,Deportivo Alavés C-Iru Bat,93015,...,5.19,5.19,0.00,0.00,0.00,0.59,0.00,0.00,0.00,1.12
11760,22,Hodei Barbarin,287017,LB,SD Eibar B,0,23/24,Jornada 19,CD Lagun Onak-SD Eibar B,123658,...,5.19,5.19,0.00,0.00,0.00,0.59,0.00,0.00,0.00,1.10
7739,14,Victor Parada,178288,LB,Deportivo Alavés B,0,23/24,Jornada 29,Deportivo Alavés B-U.D. Mutilvera,130904,...,5.19,5.19,0.00,0.00,0.00,0.59,0.00,0.00,0.00,1.10
12990,24,Galhardo Urdapilleta Tiago,551164,LB,Deportivo Alavés C,0,24/25,Jornada 14,Deportivo Alavés C-Pasaia KE,158602,...,5.19,5.19,0.00,0.00,0.00,0.59,0.00,0.00,0.00,1.10


## Escalar de (0 a 9.5) para ver los mejores por posicion  "Rendimiento de la muestra" 

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [21]:

# Definir todas las posiciones posibles
main_position = ['GK', 'CF', 'CM', 'RW', 'LB', 'RB', 'CB', 'CAM', 'LW', 'CDM', 'RM', 'LM', 'LWB', 'RWB', 'LF', 'RF', 'FW']


# Definir las ponderaciones para cada posición

pesos_main_position = {
    'GK': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.2, 'KPI_progresion': 0.05,
           'KPI_habilidad_individual': 0.1, 'KPI_peligro_generado': 0.05, 'KPI_finalizacion': 0.05,
           'KPI_eficacia_defensiva': 0.8, 'KPI_juego_aereo': 0.7, 'KPI_capacidad_recuperacion': 0.5,
           'KPI_posicionamiento_tactico': 0.6},
    
    'CB': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.3, 'KPI_progresion': 0.15,
           'KPI_habilidad_individual': 0.15, 'KPI_peligro_generado': 0.1, 'KPI_finalizacion': 0.1,
           'KPI_eficacia_defensiva': 0.9, 'KPI_juego_aereo': 0.75, 'KPI_capacidad_recuperacion': 0.7,
           'KPI_posicionamiento_tactico': 0.85},

    'RB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.4, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.6, 'KPI_peligro_generado': 0.35, 'KPI_finalizacion': 0.2,
           'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
           'KPI_posicionamiento_tactico': 0.5},

    'LB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.4, 'KPI_progresion': 0.55,
           'KPI_habilidad_individual': 0.6, 'KPI_peligro_generado': 0.35, 'KPI_finalizacion': 0.2,
           'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
           'KPI_posicionamiento_tactico': 0.5},

    'RWB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.25,
            'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
            'KPI_posicionamiento_tactico': 0.5},

    'LWB': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.75,
            'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.25,
            'KPI_eficacia_defensiva': 0.45, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.6,
            'KPI_posicionamiento_tactico': 0.5},

    'CDM': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.55, 'KPI_progresion': 0.65,
            'KPI_habilidad_individual': 0.55, 'KPI_peligro_generado': 0.4, 'KPI_finalizacion': 0.2,
            'KPI_eficacia_defensiva': 0.75, 'KPI_juego_aereo': 0.6, 'KPI_capacidad_recuperacion': 0.75,
            'KPI_posicionamiento_tactico': 0.8},

    'CM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.7, 'KPI_progresion': 0.7,
           'KPI_habilidad_individual': 0.65, 'KPI_peligro_generado': 0.55, 'KPI_finalizacion': 0.3,
           'KPI_eficacia_defensiva': 0.4, 'KPI_juego_aereo': 0.3, 'KPI_capacidad_recuperacion': 0.5,
           'KPI_posicionamiento_tactico': 0.5},

    'CAM': {'KPI_condicional': 0.4, 'KPI_construccion_ataque': 0.75, 'KPI_progresion': 0.7,
            'KPI_habilidad_individual': 0.85, 'KPI_peligro_generado': 0.8, 'KPI_finalizacion': 0.6,
            'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.2,
            'KPI_posicionamiento_tactico': 0.5},

    'RM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LM': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'RW': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LW': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'LF': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'RF': {'KPI_condicional': 0.5, 'KPI_construccion_ataque': 0.6, 'KPI_progresion': 0.65,
           'KPI_habilidad_individual': 0.7, 'KPI_peligro_generado': 0.6, 'KPI_finalizacion': 0.5,
           'KPI_eficacia_defensiva': 0.2, 'KPI_juego_aereo': 0.1, 'KPI_capacidad_recuperacion': 0.3,
           'KPI_posicionamiento_tactico': 0.4},

    'CF': {'KPI_condicional': 0.6, 'KPI_construccion_ataque': 0.5, 'KPI_progresion': 0.5,
           'KPI_habilidad_individual': 0.8, 'KPI_peligro_generado': 0.85, 'KPI_finalizacion': 0.9,
           'KPI_eficacia_defensiva': 0.15, 'KPI_juego_aereo': 0.5, 'KPI_capacidad_recuperacion': 0.2,
           'KPI_posicionamiento_tactico': 0.4}
}


# Leer el archivo CSV (ajusta el nombre del archivo según sea necesario)
df_KPI_players = pd.read_parquet('../../data/archivos_parquet/df_KPI_players.parquet')

# Calcular el rendimiento ponderado total como el promedio de los KPIs
def calcular_rendimiento_promedio(df):
    kpi_columns = [
        'KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
        'KPI_habilidad_individual', 'KPI_peligro_generado', 'KPI_finalizacion', 
        'KPI_eficacia_defensiva', 'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
        'KPI_posicionamiento_tactico'
    ]
    # Calcular el promedio de los KPIs para cada jugador
    df['KPI_rendimiento'] = df[kpi_columns].mean(axis=1)

# Suponiendo que df_KPI_players es el DataFrame original con los datos de los jugadores
calcular_rendimiento_promedio(df_KPI_players)

# Normalizar los valores entre 0 y 9.5
def normalizar(valores):
    min_val = valores.min()
    max_val = valores.max()
    
    # Normalización al rango [0, 9.5]
    normalizado = ((valores - min_val) / (max_val - min_val)) * 9.5
    return normalizado

# Normalizar las columnas de KPI
kpi_columns = ['KPI_condicional', 'KPI_construccion_ataque', 'KPI_progresion', 
               'KPI_habilidad_individual', 'KPI_peligro_generado', 
               'KPI_finalizacion', 'KPI_eficacia_defensiva', 
               'KPI_juego_aereo', 'KPI_capacidad_recuperacion', 
               'KPI_posicionamiento_tactico', 'KPI_rendimiento']

for kpi in kpi_columns:
    df_KPI_players_posicion[kpi] = normalizar(df_KPI_players[kpi]).round(2)


# Mostrar el DataFrame final
df_KPI_players_normalizado= df_KPI_players_posicion.sort_values(by='KPI_rendimiento', ascending=False).round(2)
df_KPI_players_normalizado.head(25)

# Exportar el DataFrame normalizado a formato parquet
df_KPI_players_normalizado.to_parquet('../../data/archivos_parquet/df_KPI_players_normalizado.parquet')

FileNotFoundError: [Errno 2] No such file or directory: '../../data/archivos_parquet/df_KPI_players.parquet'

In [ ]:
df_KPI_players_normalizado

In [ ]:
# Ordenar el DataFrame por KPI_rendimiento de mayor a menor y obtener las primeras 25 filas
top_25_players = df_KPI_players_normalizado.sort_values(by='KPI_rendimiento', ascending=False).head(25)

# Para mostrar solo las columnas más relevantes
top_25_players = top_25_players[[
    'player', 
    'team_name',
    'main_position',
    'league_name',
    'season_name',
    'KPI_rendimiento'
]]

# Mostrar los resultados
print(top_25_players)